In [1]:
from osgeo import gdal
import os

# 矢量数据的路径
tif = r"F:\20230426校园\result.tif"

In [2]:
def printInfo(tif_path):
    tif = gdal.Open(tif_path)
    # 输出栅格的详细信息
    print("栅格的详细信息：")
    print(tif)
    # 输出栅格的宽度
    print("栅格的宽度：")
    print(tif.RasterXSize)
    # 输出栅格的高度
    print("栅格的高度：")
    print(tif.RasterYSize)
    # 输出栅格的波段数
    print("栅格的波段数：")
    print(tif.RasterCount)
    # 输出栅格的投影信息
    print("栅格的投影信息：")
    print(tif.GetProjection())
    # 输出栅格的地理变换信息
    print("栅格的地理变换信息：")
    print(tif.GetGeoTransform())
    # 输出各个波段的信息
    for i in range(tif.RasterCount):
        band = tif.GetRasterBand(i + 1)
        print("波段{}的最大值：".format(i + 1))
        print(band.GetMaximum())
        print("波段{}的最小值：".format(i + 1))
        print(band.GetMinimum())
        print("波段{}的NoData值：".format(i + 1))
        print(band.GetNoDataValue())
    return tif.ReadAsArray().astype(int)
tif_array = printInfo(tif)
print(tif_array.shape)

栅格的详细信息：
<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000002806627C360> >
栅格的宽度：
10121
栅格的高度：
13015
栅格的波段数：
4
栅格的投影信息：
GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]
栅格的地理变换信息：
(114.29686812859407, 1.1946778073479436e-06, 0.0, 34.82429582911058, 0.0, -9.852145410604862e-07)
波段1的最大值：
255.0
波段1的最小值：
11.0
波段1的NoData值：
0.0
波段2的最大值：
254.0
波段2的最小值：
16.0
波段2的NoData值：
0.0
波段3的最大值：
254.0
波段3的最小值：
4.0
波段3的NoData值：
0.0
波段4的最大值：
255.0
波段4的最小值：
32.0
波段4的NoData值：
0.0
(4, 13015, 10121)


In [3]:
# 第一波段为Red，第二波段为Green，第三波段为Blue，第四波段为NIR
# 计算NDVI
def add_NDVI(tif_array):
    print("正在计算NDVI...")
    tif_array = tif_array.astype(float)
    # 创建一个和tif_array一样大小的数组，用来存放NDVI
    NDVI = tif_array[0].copy()
    for i in range(tif_array.shape[1]):
        for j in range(tif_array.shape[2]):
            if tif_array[3][i][j] + tif_array[0][i][j] == 0:
                NDVI[i][j] = 0
            else:
                NDVI[i][j] = (tif_array[3][i][j] - tif_array[0][i][j]) / (tif_array[3][i][j] + tif_array[0][i][j])
    return NDVI

NDVI = add_NDVI(tif_array)

# 输出NDVI的最大值
print("NDVI的最大值：")
print(NDVI.max())
# 输出NDVI的最小值
print("NDVI的最小值：")
print(NDVI.min())

正在计算NDVI...
NDVI的最大值：
1.0
NDVI的最小值：
0.0


In [4]:
import numpy as np
def add_NDVI_optimized(tif_array):
    print("正在计算NDVI...")
    # 将tif_array转换为int类型
    tif_array = tif_array.astype(float)
    # 计算红外波段和红色波段的像素值之和
    sum_bands = tif_array[3, :, :] + tif_array[0, :, :]
    NDVI = np.zeros_like(tif_array[0, :, :])
    # 计算NDVI
    NDVI[sum_bands == 0] = 0
    NDVI[sum_bands != 0] = (tif_array[3, :, :][sum_bands != 0] - tif_array[0, :, :][sum_bands != 0]) / sum_bands[sum_bands != 0]
    return NDVI

NDVI_optimized = add_NDVI_optimized(tif_array)
# 输出NDVI的最大值
print("NDVI的最大值：")
print(NDVI_optimized.max())
# 输出NDVI的最小值
print("NDVI的最小值：")
print(NDVI_optimized.min())

正在计算NDVI...


NDVI的最大值：
1.0
NDVI的最小值：
0.0


In [5]:
# 对比两个ndvi的计算结果是否一致
print("两个NDVI计算结果是否一致：")
print(np.allclose(NDVI, NDVI_optimized))


两个NDVI计算结果是否一致：
True


In [6]:
# 将计算出的NDVI加入到tif_array中
tif_array = np.concatenate((tif_array, NDVI.reshape(1, tif_array.shape[1], tif_array.shape[2])), axis=0)
print(tif_array.shape)


(5, 13015, 10121)


In [7]:
# 保存为新的tif文件
def save_tif(tif_array, tif_path, save_path):
    # 获取原始tif文件的信息
    tif = gdal.Open(tif_path)
    # 获取原始tif文件的波段数
    bands = tif.RasterCount
    # 获取原始tif文件的地理变换信息
    geotransform = tif.GetGeoTransform()
    # 获取原始tif文件的投影信息
    projection = tif.GetProjection()
    # 获取原始tif文件的宽度
    width = tif.RasterXSize
    # 获取原始tif文件的高度
    height = tif.RasterYSize
    # 创建一个新的tif文件
    driver = gdal.GetDriverByName("GTiff")
    # 创建一个和原始tif文件一样大小的tif文件
    new_tif = driver.Create(save_path, width, height, bands + 1, gdal.GDT_Float32)
    # 将tif_array中的数据写入到新的tif文件中
    for i in range(bands + 1):
        new_tif.GetRasterBand(i + 1).WriteArray(tif_array[i])
    # 将地理变换信息写入到新的tif文件中
    new_tif.SetGeoTransform(geotransform)
    # 将投影信息写入到新的tif文件中
    new_tif.SetProjection(projection)
    # 释放资源
    del new_tif
    print("新的tif文件保存成功！")
save_tif(tif_array, tif, r"F:\20230426校园\result_with_ndvi.tif")

新的tif文件保存成功！


In [15]:
import arcpy

arcpy.env.overwriteOutput = True

shp_path = r"F:\20230426校园\label.shp"

out_raster_path = r"F:\20230426校园\label02.tif"
# 输出矢量数据的字段
arcpy.ListFields(shp_path)
for field in arcpy.ListFields(shp_path):
    print(field.name)
# 增加字段
arcpy.CalculateField_management(shp_path,'Num',"'int(!Class!)'", 'Python3')
# 将shp文件转换为栅格文件
arcpy.FeatureToRaster_conversion(in_features=shp_path, field="Num", out_raster=out_raster_path, cell_size=1)

print("shp文件转换为栅格文件成功！")

FID
Shape
Id
Class
Num
shp文件转换为栅格文件成功！
